In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime, timedelta

In [58]:
# function to add the seconds of the row to the timestamp of the row 

def addSecondsToTimestamp(timestamp, secondsToAdd):
    return timestamp + timedelta(seconds=secondsToAdd)


def formDataset(path):
    convert_to_timestamp = lambda x: datetime.fromtimestamp(int(x))

    dataset = pd.read_csv(path, delimiter=',', parse_dates=['SubmitTime'], date_parser=convert_to_timestamp)

    dataset['StopTime'] = dataset[['SubmitTime', 'RunTime']].apply(lambda x: addSecondsToTimestamp(*x), axis=1)

    #drop columns with 0 variance
    dataset.drop(["Status", "PartitionID"], inplace=True, axis=1)
    dataset = dataset.loc[:, dataset.apply(pd.Series.nunique) != 1]
    #drop columns with type object
    dataset.drop(dataset.select_dtypes(['object']), inplace=True, axis=1)

    try:
        cancelled_jobs = dataset[dataset["ReqTime"] < dataset["RunTime"]]
        dataset = dataset[ ~ (dataset["ReqTime"] < dataset["RunTime"])]

        #drop columns with 'ReqTime' == -1
        cancelled_jobs.drop(cancelled_jobs["ReqTime"] == -1, inplace=True)
        cancelled_jobs.to_csv(f"./datasets/dropped/dropped-{path.split('/')[-1]}", index=False)
    except:
        pass



    dataset.to_csv( f"./datasets/formed/formed-{path.split('/')[-1]}", index=False)

In [59]:
for csv in glob.glob(pathname="./datasets/raw/*.csv"):
    print(csv)
    if csv in ['./datasets/raw/fastStorage.csv', './datasets/raw/rnd.csv']:
        continue
    formDataset(csv)

./datasets/raw/grid5000.csv
./datasets/raw/das2.csv


/var/folders/rz/x38ry1m901jcjct47ydtrgph0000gn/T/ipykernel_96133/3726267829.py:10: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(path, delimiter=',', parse_dates=['SubmitTime'], date_parser=convert_to_timestamp)


./datasets/raw/rnd.csv
./datasets/raw/nordugrid.csv
./datasets/raw/auvergrid.csv
./datasets/raw/fastStorage.csv
./datasets/raw/sharcnet.csv


In [64]:
df = pd.read_csv("./datasets/raw/sharcnet.csv")
df.head(10)

,JobID,SubmitTime,WaitTime,RunTime,NProc,UsedCPUTime,UsedMemory,ReqNProcs,ReqTime,ReqMemory,...,JobStructureParams,UsedNetwork,UsedLocalDiskSpace,UsedResources,ReqPlatform,ReqNetwork,ReqLocalDiskSpace,ReqResources,VOID,ProjectID
0,1,1135130133,0,6,1,0.10,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
1,2,1135130401,1,13,1,0.13,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
2,3,1135130415,0,17,20,0.01,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
3,4,1135130438,0,17,1,0.16,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
4,5,1135130471,0,5,4,0.03,256.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
5,6,1135130835,0,16,1,0.13,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
6,7,1138120593,0,6,1,0.11,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
7,8,1138120603,0,5,1,0.10,1024.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
8,9,1138120664,0,5,1,0.13,1024.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1
9,10,1138120938,0,6,80,0.00,-1.0,-1,-1.0,-1.0,...,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1


In [65]:
df.dtypes

JobID                   int64
SubmitTime              int64
WaitTime                int64
RunTime                 int64
NProc                   int64
UsedCPUTime           float64
UsedMemory            float64
ReqNProcs               int64
ReqTime               float64
ReqMemory             float64
Status                  int64
UserID                 object
GroupID                 int64
ExecutableID           object
QueueID                 int64
PartitionID             int64
OrigSiteID             object
LastRunSiteID          object
JobStructure            int64
JobStructureParams      int64
UsedNetwork           float64
UsedLocalDiskSpace    float64
UsedResources           int64
ReqPlatform             int64
ReqNetwork            float64
ReqLocalDiskSpace     float64
ReqResources            int64
VOID                    int64
ProjectID               int64
dtype: object

In [66]:
df.describe()

,JobID,SubmitTime,WaitTime,RunTime,NProc,UsedCPUTime,UsedMemory,ReqNProcs,ReqTime,ReqMemory,...,JobStructureParams,UsedNetwork,UsedLocalDiskSpace,UsedResources,ReqPlatform,ReqNetwork,ReqLocalDiskSpace,ReqResources,VOID,ProjectID
count,1.195242e+06,1.195242e+06,1.195242e+06,1.195242e+06,1.195242e+06,1.195242e+06,1.195242e+06,1195242.0,1195242.0,1195242.0,...,1195242.0,1195242.0,1195242.0,1195242.0,1195242.0,1195242.0,1195242.0,1195242.0,1195242.0,1195242.0
mean,5.976215e+05,1.157647e+09,2.866688e+04,3.165430e+04,2.993179e+00,2.075724e+04,8.049621e+04,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
std,3.450368e+05,7.924207e+06,8.291687e+04,1.165614e+05,2.455266e+01,5.154719e+06,4.639791e+05,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000e+00,1.135130e+09,-1.000000e+00,-1.000000e+00,-1.000000e+00,-2.124628e+09,-1.000000e+00,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
25%,2.988112e+05,1.151855e+09,8.000000e+00,9.900000e+01,1.000000e+00,1.900000e-01,4.096000e+03,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
50%,5.976215e+05,1.159679e+09,1.119000e+03,2.789000e+03,1.000000e+00,2.114500e+02,7.168000e+03,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
75%,8.964318e+05,1.164127e+09,1.913500e+04,2.191000e+04,1.000000e+00,9.718000e+03,2.355200e+04,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
max,1.195242e+06,1.168908e+09,1.916568e+06,1.390840e+07,3.000000e+03,2.087029e+09,3.202150e+07,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [61]:
df1 = df[df["ReqTime"]< df["RunTime"]]

In [62]:
df1["Diff"] = - df["ReqTime"] + df["RunTime"]

/var/folders/rz/x38ry1m901jcjct47ydtrgph0000gn/T/ipykernel_96133/3406249500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Diff"] = - df["ReqTime"] + df["RunTime"]


In [63]:
df1.describe()

,JobID,SubmitTime,WaitTime,RunTime,NProc,UsedCPUTime,UsedMemory,ReqNProcs,ReqTime,ReqMemory,...,UsedNetwork,UsedLocalDiskSpace,UsedResources,ReqPlatform,ReqNetwork,ReqLocalDiskSpace,ReqResources,VOID,ProjectID,Diff
count,1878.000000,1.878000e+03,1878.000000,1.878000e+03,1878.0,1878.000000,1.878000e+03,1878.0,1878.000000,1.878000e+03,...,1878.0,1878.0,1878.0,1878.0,1878.0,1878.0,1878.0,1878.0,1878.0,1.878000e+03
mean,198466.585197,1.153133e+09,11011.666667,2.057195e+05,1.0,26474.198616,3.652458e+05,1.0,192311.789137,2.415409e+04,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.340775e+04
std,125734.421505,9.161548e+06,29556.968513,1.594961e+05,0.0,37192.910675,3.122885e+05,0.0,107357.919293,1.368525e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.144263e+05
min,35.000000,1.136284e+09,0.000000,6.040000e+02,1.0,0.000000,0.000000e+00,1.0,600.000000,-1.000000e+00,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000e+00
25%,84733.500000,1.144955e+09,1.000000,8.008600e+04,1.0,24.000000,7.952200e+04,1.0,79980.000000,-1.000000e+00,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.400000e+01
50%,166247.000000,1.155411e+09,246.500000,2.592440e+05,1.0,3087.000000,3.520400e+05,1.0,259200.000000,-1.000000e+00,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.700000e+01
75%,328341.250000,1.160560e+09,7186.750000,2.592908e+05,1.0,54109.250000,5.900760e+05,1.0,259200.000000,-1.000000e+00,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.010000e+02
max,402277.000000,1.167325e+09,442804.000000,1.575814e+06,1.0,238553.000000,2.523348e+06,1.0,345600.000000,1.126400e+06,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.425234e+06
